# Avaliação – Produto Renda Fixa
## Construção de uma calculadora de renda fixa em Python - NTN-F


Formato de envio: arquivo .ipynb (Jupyter Notebook) conforme instruções a seguir.

- Pode construir quantos métodos/classes/funções adicionais desejarem;
- Pode utilizar bibliotecas abertas;
- O arquivo deve conter as funções solicitadas e os cenários de teste (pelo menos um teste para cada função/método). 
- Comente as funções e o que mais achar necessário no código;

Exemplo de definição da função com docstring detalhando os parâmetros:

```{python}
def calcula_soma(a, b):
''' Calcula a soma de dois números decimais
    a: float
    b: float
    '''
    return a + b
```

Exemplo de teste da função com a definição dos parâmetros e chamada da função:

```{python}
a = 1.3
b = 10.07
resultado = calcula_soma(a, b):
print(f'Resultado: {resultado}')
```

- A nota da avaliação considerará apenas o resultado das funções desejadas (existência de bugs, bom funcionamento, etc);
- Na avaliação qualitativa, faremos comentários de melhorias na estrutura do código (para fins de evolução de aprendizado, e não para reduzir nota).

### Funções/métodos mínimos de entrega:
- calcula_prazo(dt_ini, dt_fim, feriados, convencao)
    - Retorna: prazo anualizado (float)
    - convencao: 'DU/252' ou 'DC/360'
    
    
- constroi_fluxo(dt_fim, frequencia)
    - Retorna: Lista de datas dos fluxos (list datetime.date)
    
    
- calcula_pu(VF, prazo_anual, taxa_anual)
    - Retorna pu (float)
    
    
- calcula_taxa_anual(PU, prazo_anual, valor_base=100)
    - Retorna: taxa_anual (float)
    
    
- calcula_pu_ntnf(dt_venc, dt_base, tir) *
    - Retorna: pu (float)
    - Imprime tabela com o cashflow (Data do fluxo, VF, DU, Fator de desconto, PU)

\* semelhante ao calculado em aula no Excel. Sugere-se aproveitar as funções anteriores para este cálculo.

### Sugestões (itens não obrigatórios):
- Utilizar variáveis de data no formato datetime.date;
- Utilizar o calendário Anbima em .xls (disponibilizado no material de aula ou em: https://www.anbima.com.br/feriados/arqs/feriados_nacionais.xls) para cálculo de dias úteis;

### Itens adicionais:

Se você deseja incrementar seu framework para construção de portfólio pessoal, seguem algumas sugestões de melhorias para o projeto. 

Obs: Estes itens não serão considerados na nota, trabalhe neles apenas após ter garantido a parte obrigatória:

- Implementar diversas contagens de prazo. Ex: DU/252, 30/360, ACT/360, ACT/ACT. Ver padrões ISDA.
- Implementar tratamento de dias úteis/feriados. Por exemplo, se o vencimento de um fluxo cai num feriado/fim de semana, deslocar para o dia útil anterior ou próximo; escolher o critério via parâmetro;
- Calcular accrual. Ex: def calcula_accrual(dt_ini, dt_base, taxa_anual);
- Ler dados históricos (por exemplos, planilhas do tesouro direto) e fazer gráfico da série de preços de mercado ou das taxas;
- Fazer gráficos de séries históricas de preço de mercado e preço accruado no mesmo gráfico.
- Etc.


In [1]:
# Função datetime.timedelta

- calcula_prazo(dt_ini, dt_fim, feriados, convencao)
    - Retorna: prazo anualizado (float)
    - convencao: 'DU/252' ou 'DC/360'
    

## Criando função de cálculo de prazo

In [9]:
import pandas as pd
import numpy as np
from datetime import datetime

def calcula_prazo(dt_ini, dt_fim, feriados, convencao):
    f = '%d/%m/%Y'
    dif = (datetime.strptime(dt_fim, f) - datetime.strptime(dt_ini, f)).days
    dif_uteis = np.busday_count(datetime.strptime(dt_ini, f).date(), datetime.strptime(dt_fim, f).date()) - feriados
    if convencao == 252:
        return dif_uteis/252
    elif convencao == 360:
        return dif/360

In [12]:
s = '01/01/2022'
t = '31/01/2022'
f = '%d/%m/%Y'

In [13]:
# Teste

calcula_prazo(s, t, 3, 252)

0.06746031746031746

In [14]:
from datetime import datetime
dif = (datetime.strptime(t, f) - datetime.strptime(s, f))
dif.days

30

## Criando função para mostrar a lista de datas entre duas datas

In [15]:
from datetime import date, timedelta

d1 = datetime.strptime(s, f)
d2 = datetime.strptime(t, f)

In [7]:
# Padronizando as datas

tempo = '%Y-%m-%d %H:%M:%S'

dias = pd.Series(pd.date_range(d1, d2-timedelta(days=1),freq='d').strftime('%Y-%m-%d %H:%M:%S'))

dias.head()

0    2022-01-01 00:00:00
1    2022-01-02 00:00:00
2    2022-01-03 00:00:00
3    2022-01-04 00:00:00
4    2022-01-05 00:00:00
dtype: object

In [8]:
type(dias)

pandas.core.series.Series

In [9]:
dias.size

30

## Importando a planilha de feriados nacionais e fazendo o tratamento

### Função Tratamento de Datas

In [12]:
import pandas as pd

df_fn = pd.read_excel('feriados_nacionais.xls')
df_fn.sample(10).head()

,Data,Dia da Semana,Feriado
313,2027-02-08 00:00:00,segunda-feira,Carnaval
878,2074-02-27 00:00:00,terça-feira,Carnaval
678,2057-06-21 00:00:00,quinta-feira,Corpus Christi
348,2030-01-01 00:00:00,terça-feira,Confraternização Universal
38,2004-02-24 00:00:00,terça-feira,Carnaval


In [13]:
df_fn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 945 entries, 0 to 944
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Data           941 non-null    object
 1   Dia da Semana  936 non-null    object
 2   Feriado        936 non-null    object
dtypes: object(3)
memory usage: 22.3+ KB


In [14]:
import pandas as pd

def clean_data(df):
    df = pd.to_datetime(df, errors='coerce')
    df = df.dropna(how = 'all')
    df = df.dt.strftime('%Y-%m-%d %H:%M:%S')

    return df

In [15]:
# Teste

lista_feriados = clean_data(df_fn['Data'])
lista_feriados.size

936

In [16]:
lista_feriados.head()

0    2001-01-01 00:00:00
1    2001-02-26 00:00:00
2    2001-02-27 00:00:00
3    2001-04-13 00:00:00
4    2001-04-21 00:00:00
Name: Data, dtype: object

### Função Lista com as Datas dos feriados entre o intervalo

In [17]:
def datas_feriados(data_dias, data_feriados):
    dias_feriados = pd.Series(data_dias.isin(data_feriados).values)
    dias_feriados[dias_feriados].index
    feriados = pd.Series(data_dias[dias_feriados].values)
    return feriados

In [18]:
# Teste

f_entre = datas_feriados(dias, lista_feriados)
f_entre

0    2022-01-01 00:00:00
dtype: object

In [19]:
# Número de Feriados

f_entre.count()

1

### Função para Lista de Dias sem os feriados

In [20]:
def sem_feriados(data_dias, data_feriados):
    lista_s_feriados = data_dias[~data_dias.isin(data_feriados)]
    return lista_s_feriados

In [21]:
# Teste

lista_sem_feriados = sem_feriados(dias, f_entre)
lista_sem_feriados

1     2022-01-02 00:00:00
2     2022-01-03 00:00:00
3     2022-01-04 00:00:00
4     2022-01-05 00:00:00
5     2022-01-06 00:00:00
6     2022-01-07 00:00:00
7     2022-01-08 00:00:00
8     2022-01-09 00:00:00
9     2022-01-10 00:00:00
10    2022-01-11 00:00:00
11    2022-01-12 00:00:00
12    2022-01-13 00:00:00
13    2022-01-14 00:00:00
14    2022-01-15 00:00:00
15    2022-01-16 00:00:00
16    2022-01-17 00:00:00
17    2022-01-18 00:00:00
18    2022-01-19 00:00:00
19    2022-01-20 00:00:00
20    2022-01-21 00:00:00
21    2022-01-22 00:00:00
22    2022-01-23 00:00:00
23    2022-01-24 00:00:00
24    2022-01-25 00:00:00
25    2022-01-26 00:00:00
26    2022-01-27 00:00:00
27    2022-01-28 00:00:00
28    2022-01-29 00:00:00
29    2022-01-30 00:00:00
dtype: object

In [22]:
# Número de dias sem feriados

lista_sem_feriados.count()

29

In [23]:
# Conferindo o resultado

dias.count() - f_entre.count()

29

- constroi_fluxo(dt_fim, frequencia)
    - Retorna: Lista de datas dos fluxos (list datetime.date)


## Função Lista de Datas dos Fluxos

### Pagamentos ocorrem em 01/01 e 01/07

In [40]:
def constroi_fluxo(dt_fm, frequencia):
    f = '%d/%m/%Y'
    d1 = date.today()
    d2 = datetime.strptime(dt_fm, f)
    dias_f = pd.Series(pd.date_range(d1, d2-timedelta(days=1), 
                                     freq=frequencia))
    return dias_f

In [41]:
# Teste

constroi_fluxo('28/06/2023', 'QS')

0   2022-04-01
1   2022-07-01
2   2022-10-01
3   2023-01-01
4   2023-04-01
dtype: datetime64[ns]

In [42]:
constroi_fluxo('28/06/2023', 'QS')[1]

Timestamp('2022-07-01 00:00:00')

In [77]:
def data_semestre(s):
    for i in s:
        if i.month in [1,7]:
            pass
        else:
            s = s.drop(s[s == i].index[0])
    return s#.apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))

In [78]:
s = constroi_fluxo('28/06/2023', 'QS')

In [79]:
data_semestre(s)

1   2022-07-01
3   2023-01-01
dtype: datetime64[ns]

### - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

- calcula_pu(VF, prazo_anual, taxa_anual)
    - Retorna pu (float)


## Função Cálculo do PU

In [ ]:
def calcula_pu(VF, prazo_anual, taxa_anual):
    PU = VF / (1 + taxa_anual)**prazo_anual
    return PU

In [ ]:
calcula_pu(1000, 5.95/100, 15/252)

In [ ]:
# Teste

VF = 1000
i = 5.95/100
n = 15/252

PU = VF / (1 + i)**n
PU

- calcula_taxa_anual(PU, prazo_anual, valor_base=100)
    - Retorna: taxa_anual (float)
    

## Taxa Anual

In [166]:
def calcula_taxa_anual(PU, prazo_anual, convencao):    
    VN = 1000
    a = (VN/PU)
    if convencao == 252:
        return (a ** (1/(prazo_anual*252)) - 1)/100
    elif convencao == 360:
        return (a ** (1/(prazo_anual*360)) - 1)/100

In [167]:
# Teste

calcula_taxa_anual(959, 5, 360)

2.325816163284422e-07

- calcula_pu_ntnf(dt_venc, dt_base, tir) *
    - Retorna: pu (float)
    - Imprime tabela com o cashflow (Data do fluxo, VF, DU, Fator de desconto, PU)

## Função que Cálcula o PU de NTN-F

In [155]:
def calcula_pu_ntnf(dt_venc, dt_base, tir):
    f = '%d/%m/%Y'
    d1 = datetime.strptime(dt_base, f)
    d2 = datetime.strptime(dt_venc, f)
    intervalo = pd.Series(pd.date_range(d1, d2-timedelta(days=1), 
                                     freq=frequencia))
    PU = VN / (1 + tir)**intervalo
    return PU

In [19]:
def calcula_pu_ntnf(dt_venc, dt_base, tir):
    VN = 1000
    f = '%d/%m/%Y'
    d1 = datetime.strptime(dt_base, f)
    d2 = datetime.strptime(dt_venc, f)
    intervalo = pd.Series(pd.date_range(d1, d2-timedelta(days=1), 
                                     freq='D'))
    tir_s = ((1+(tir/100))**(1/2) - 1)
    
    PU = VN / ((1 + tir)**(len(intervalo)))
               
    return PU

In [20]:
f = '%d/%m/%Y'
d1 = datetime.strptime('18/02/2022', f)
d2 = datetime.strptime('01/07/2024', f)
intervalo = pd.Series(pd.date_range(d1, d2-timedelta(days=1), 
                                     freq='D'))
    
len(intervalo)

864

In [22]:
1000/((1+(11.66/100))**(864/252))

685.140399419581

In [31]:
# Cupom semestral

1000*((1+(10.75/100))**(1/2)-1)

52.378258992459294

In [122]:
685.140399419581 + (56.692954457443044*2)

798.526308334467

In [89]:
calcula_pu_ntnf('01/07/2024', '18/02/2022', 11.66/100)

767.4422479321714

In [ ]:
pv(rate, nper, pmt[, fv, when])

In [24]:
pip install numpy-financial

Note: you may need to restart the kernel to use updated packages.


In [34]:
import numpy_financial as npf
npf.pv(0.056692954457443046, 2, -52.378258992459294)

96.47680121488293

In [38]:
1000/((1+(11.66/100))**(864/252))

685.140399419581

In [39]:
96.47680121488293 + 685.140399419581

781.6172006344639

R$ 772,76

*TESOURO PREFIXADO 2024*

11,66%	R$ 772,76	01/07/2024


*TESOURO PREFIXADO 2026*

11,32%	R$ 661,24	01/01/2026


TESOURO PREFIXADO com juros semestrais 2031

11,48%	R$ 937,85	01/01/2031

In [129]:
((1+(11.66/100))**(1/2) - 1)

0.056692954457443046

In [161]:
(1000*(((1+0.1)**(1/2))-1))

48.808848170151634

In [30]:
# Cupons à valor presente

((1000*((1+0.1)**(1/2) - 1))/((1+0.1298)**(120/252))) + ((1000*((1+0.1)**(1/2) - 1))/((1+0.1298)**(248/252))) + ((1000*((1+0.1)**(1/2) - 1))/((1+0.1298)**(372/252))) + ((1000*((1+0.1)**(1/2) - 1))/((1+0.1298)**(499/252)))


168.43125061860943

In [28]:
# Valor de Face

1000/((1+0.1298)**(499/252))

785.323290594915

In [27]:
785.323290594915+168.43125061860943

953.7545412135244

### PU de compra = R$ 953,76

PU =1.000,00*[(1+10%)12 −1]+1.000,00*[(1+10%)12 −1]+1.000,00*[(1+10%)12 −1]+1.000,00*[(1+10%)12] (1+12,98%)120/252 (1+12,98%)248/252 (1+12,98%)372/252 (1+12,98%)499/252

In [59]:
data_semestre(s)

1   2022-07-01
3   2023-01-01
dtype: datetime64[ns]

In [106]:
f = '%d/%m/%Y'
d1 = datetime.strptime('18/02/2022', f)
deltas_s = []

for i in data_semestre(s):
    deltas_s.append((i - d1).days)
                                       
deltas_s

[133, 317]

In [108]:
# Função com lista de Delta entre Data_base e Datas dos Cupons

def deltas_cupom(dt_venc, dt_base, datas_cupom):
    f = '%d/%m/%Y'
    d1 = datetime.strptime(dt_base, f)
    deltas_s = []

    for i in datas_cupom:
        deltas_s.append((i - d1).days)

    return deltas_s

In [109]:
deltas_cupom('28/06/2023', '18/02/2022', data_semestre(s))

[133, 317]

In [71]:
data_semestre(s)[1]

Timestamp('2022-07-01 00:00:00')

In [104]:
(data_semestre(s)[1] - d1).days

133

In [102]:
(data_semestre(s)[3] - d1).days

317

### Teste LTN

In [56]:
# Número de dias

lista_dias('18/02/2022', '01/07/2024')

0      2022-02-18
1      2022-02-19
2      2022-02-20
3      2022-02-21
4      2022-02-22
          ...    
859    2024-06-26
860    2024-06-27
861    2024-06-28
862    2024-06-29
863    2024-06-30
Length: 864, dtype: object

In [57]:
lista_dias('18/02/2022', '01/07/2024').count()

864

In [58]:
# Datas que são feriados entre os dias

datas_feriados(lista_dias('18/02/2022', '01/07/2024'), lista_feriados)

0     2022-02-28
1     2022-03-01
2     2022-04-15
3     2022-04-21
4     2022-05-01
5     2022-06-16
6     2022-09-07
7     2022-10-12
8     2022-11-02
9     2022-11-15
10    2022-12-25
11    2023-01-01
12    2023-02-20
13    2023-02-21
14    2023-04-07
15    2023-04-21
16    2023-05-01
17    2023-06-08
18    2023-09-07
19    2023-10-12
20    2023-11-02
21    2023-11-15
22    2023-12-25
23    2024-01-01
24    2024-02-12
25    2024-02-13
26    2024-03-29
27    2024-04-21
28    2024-05-01
29    2024-05-30
dtype: object

In [59]:
datas_feriados(lista_dias('18/02/2022', '01/07/2024'), lista_feriados).count()

30

In [60]:
# Lista sem feriados
sem_feriados(lista_dias('18/02/2022', '01/07/2024'),
             datas_feriados(lista_dias('18/02/2022', '01/07/2024'), lista_feriados))

0      2022-02-18
1      2022-02-19
2      2022-02-20
3      2022-02-21
4      2022-02-22
          ...    
859    2024-06-26
860    2024-06-27
861    2024-06-28
862    2024-06-29
863    2024-06-30
Length: 834, dtype: object

In [61]:
ltn_sf = sem_feriados(lista_dias('18/02/2022', '01/07/2024'),
             datas_feriados(lista_dias('18/02/2022', '01/07/2024'), lista_feriados)).count()

ltn_sf

834

In [62]:
f = '%d/%m/%Y'

np.busday_count(datetime.strptime('18/02/2022', f).date(), datetime.strptime('01/07/2024', f).date())

616

In [63]:
calcula_prazo('18/02/2022','01/07/2024', ltn_sf, 252)

-0.8650793650793651

In [64]:
1000/((1+0.1166)**((616-30)/252))

773.782704751225

R$ 772,76

### - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

In [ ]:
import pandas as pd

def clean_data(df):
    df = pd.to_datetime(df, errors='coerce')
    df = df.dropna(how = 'all')
    df = df.dt.strftime('%Y-%m-%d %H:%M:%S')

    return df

In [ ]:
teste = clean_data(df_fn['Data'])
teste.size

In [ ]:
teste.head()

In [ ]:
teste.dt.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
df_fn['Data'] = pd.to_datetime(df_fn['Data'], errors='coerce')

In [ ]:
df_fn = df_fn.dropna(how = 'all')
df_fn

In [ ]:
df_fn.info()

In [ ]:
dias_comum = pd.Series(dias.isin(lista_feriados).values)
dias_comum

In [ ]:
dias_comum[dias_comum].index

In [ ]:
dias[dias_comum]

In [ ]:
feriados = pd.Series(dias[dias_comum].values)
feriados

In [ ]:
a = 0

for i in dias.isin(teste).values:
    if i == True:
        a += 1
print(a)

In [ ]:
a = -1

for i in dias.isin(teste).values:
    a += 1
    print(a)

In [ ]:
dias.isin(teste).values

In [ ]:
dias.isin(teste).value_counts()

In [ ]:
df_fn['Data'].count()

### - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

In [ ]:
type(df_fn.iloc[936]['Data']) == str

In [ ]:
type(df_fn.iloc[936]['Data'])

In [ ]:
type(feriados)

In [ ]:
feriados[feriados == 'Fonte: ANBIMA'].index.values

In [ ]:
feriados[feriados == 'Fonte: ANBIMA'].keys().values

In [ ]:
from datetime import date, timedelta

d1 = datetime.strptime(s, f)
d2 = datetime.strptime(t, f)

In [ ]:
d1

In [ ]:
d1.to_pydatetime()

In [ ]:
dias_s = pd.Series(pd.date_range(d1, d2-timedelta(days=1),freq='d').strftime('%Y-%m-%d %H:%M:%S'))

dias_s.size

In [ ]:
dias_s.head()

In [ ]:
dias_c = pd.Series(pd.date_range(d1, d2,freq='d').strftime('%Y-%m-%d %H:%M:%S'))

dias_c.size

In [ ]:
dias_c

In [ ]:
feriados = feriados.map(dias_c.value_counts())

### - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

In [148]:
s.apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))

1    2022-07-01 00:00:00
3    2023-01-01 00:00:00
dtype: object

In [123]:
constroi_fluxo('28/06/2023', 'QS')[1].month

7

In [129]:
constroi_fluxo('28/06/2023', 'QS')[constroi_fluxo('28/06/2023', 'QS') == '2022-04-01 00:00:00'].index[0]

0

In [77]:
constroi_fluxo('28/06/2023', 'QS').drop(constroi_fluxo('28/06/2023', 'QS')[constroi_fluxo('28/06/2023', 'QS') == '2022-04-01 00:00:00'].index[0])

1    2022-07-01 00:00:00
2    2022-10-01 00:00:00
3    2023-01-01 00:00:00
4    2023-04-01 00:00:00
dtype: object

In [82]:
for i in constroi_fluxo('28/06/2023', 'QS'):
    if i.find('01-01') == -1:
        constroi_fluxo('28/06/2023', 'QS').drop(constroi_fluxo('28/06/2023', 'QS')
                                                [constroi_fluxo('28/06/2023', 'QS') == i].index[0])
    elif i.find('01-01') == -1:
        constroi_fluxo('28/06/2023', 'QS').drop(constroi_fluxo('28/06/2023', 'QS')
                                                [constroi_fluxo('28/06/2023', 'QS') == i].index[0])
    else:
        continue
        
print(constroi_fluxo('28/06/2023', 'QS'))

0    2022-04-01 00:00:00
1    2022-07-01 00:00:00
2    2022-10-01 00:00:00
3    2023-01-01 00:00:00
4    2023-04-01 00:00:00
dtype: object


In [27]:
date.today()

datetime.date(2022, 2, 16)

In [40]:
d2 = datetime.strptime('28/06/2023', f)
dias = pd.Series(pd.date_range(d1, d2-timedelta(days=0),freq='d').strftime('%Y-%m-%d %H:%M:%S'))

In [41]:
dias

0      2022-01-01 00:00:00
1      2022-01-02 00:00:00
2      2022-01-03 00:00:00
3      2022-01-04 00:00:00
4      2022-01-05 00:00:00
              ...         
539    2023-06-24 00:00:00
540    2023-06-25 00:00:00
541    2023-06-26 00:00:00
542    2023-06-27 00:00:00
543    2023-06-28 00:00:00
Length: 544, dtype: object

In [ ]:
[lista_sem_feriados[x:x+100] for x in range(0, lista_sem_feriados.count(), 100)][0][0]

In [ ]:
[lista_sem_feriados[x:x+100] for x in range(0, lista_sem_feriados.count(), 100)][0][0]

In [ ]:
len([lista_sem_feriados[x:x+100] for x in range(0, lista_sem_feriados.count(), 100)])

In [ ]:
dias.head()

In [ ]:
indices = []

for i in range(len(dias)):
    if '01-01' in dias[i]:
        indices.append(dias[i])
    elif '07-01' in dias[i]:
        indices.append(dias[i])
print(indices)

In [ ]:
a_string = "one two three two one"
substring = "two"

matches = re.finditer(substring, a_string)

matches_positions = [match.start() for match in matches]

print(matches_positions)

In [ ]:
a_list = [1, 2, 3, 1]

indices = []
for i in range(len(a_list)):

    if a_list[i] == 1:
        indices.append(i)

print(indices)

In [ ]:
a_list = [1, 2, 3, 1]
indices = [index for index, element in enumerate(a_list) if element == 1]

print(indices)

### - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 